# Importing Libraries

In [1]:
from tensorflow.keras.models import model_from_json
from fer_model import get_fer_model
import tensorflow as tf
import tempfile 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from livelossplot import PlotLossesKerasTF
from tensorflow.keras.optimizers import Adam

import os
import zipfile

import tensorflow_model_optimization as tfmot
import pickle

In [2]:
import time
TIMESTAMP = round(time.time())

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Loading the CK+ Dataset

In [4]:
from data import load_rafdb
train_generator = load_rafdb("train")
test_generator = load_rafdb("test")

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.


# Load Baseline Model

In [5]:
model = get_fer_model(input_size=100, input_channels=1, out_classes=7)
model.load_weights("weights_rafdb/model_weights_1626211720.h5")

### Baseline performance

In [6]:
from evaluation import get_metrics_rafdb
print("Baseline model performance:")
get_metrics_rafdb(model, test_generator)

Baseline model performance:


{'size': 29801682,
 'acc': 0.8246414602346805,
 'f_acc': 0.8333333333333334,
 'm_acc': 0.8054443554843875,
 'caucasian_acc': 0.8192258613356018,
 'afamerican_acc': 0.8675213675213675,
 'asian_acc': 0.8302277432712215,
 'age0_acc': 0.8996960486322189,
 'age1_acc': 0.8292181069958847,
 'age2_acc': 0.8044524669073405,
 'age3_acc': 0.8585657370517928,
 'age4_acc': 0.7078651685393258}

# Apply Quantisation to the Baseline Model 

In [7]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This optimisation includes the quantisation 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Evaluate

In [8]:
from evaluation import get_metrics_quantised
metrics = get_metrics_quantised(quantized_tflite_model, test_generator, dataset="rafdb")

Evaluated on 0 results so far.
Evaluated on 100 results so far.
Evaluated on 200 results so far.
Evaluated on 300 results so far.
Evaluated on 400 results so far.
Evaluated on 500 results so far.
Evaluated on 600 results so far.
Evaluated on 700 results so far.
Evaluated on 800 results so far.
Evaluated on 900 results so far.
Evaluated on 1000 results so far.
Evaluated on 1100 results so far.
Evaluated on 1200 results so far.
Evaluated on 1300 results so far.
Evaluated on 1400 results so far.
Evaluated on 1500 results so far.
Evaluated on 1600 results so far.
Evaluated on 1700 results so far.
Evaluated on 1800 results so far.
Evaluated on 1900 results so far.
Evaluated on 2000 results so far.
Evaluated on 2100 results so far.
Evaluated on 2200 results so far.
Evaluated on 2300 results so far.
Evaluated on 2400 results so far.
Evaluated on 2500 results so far.
Evaluated on 2600 results so far.
Evaluated on 2700 results so far.
Evaluated on 2800 results so far.
Evaluated on 2900 results 

In [9]:
print("Quantised model performance:")
metrics

Quantised model performance:


{'size': 6568174,
 'acc': 0.8246414602346805,
 'f_acc': 0.8320987654320988,
 'm_acc': 0.8070456365092074,
 'caucasian_acc': 0.8192258613356018,
 'afamerican_acc': 0.8675213675213675,
 'asian_acc': 0.8302277432712215,
 'age0_acc': 0.8996960486322189,
 'age1_acc': 0.8292181069958847,
 'age2_acc': 0.8050541516245487,
 'age3_acc': 0.8565737051792829,
 'age4_acc': 0.7078651685393258}

### Store results

In [10]:
with open("logs_rafdb/quantised_model_metrics_%s" % TIMESTAMP, 
          'wb') as quantised_model_metrics_file:
        pickle.dump(metrics, quantised_model_metrics_file)